In [20]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import torch
import random
import numpy as np

torch.manual_seed(31)
random.seed(31)
np.random.seed(31)

## Preprocess the dataset

In [22]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/data/modified/train_900_equal_class_dist.csv")
train

,text,label
0,যমুনা টেলিভিশন সরকারের দালাল,1
1,আজব এক ধর্ম। অবমাননার অর্থ কি ?? কেউ বলবেন?? ম...,1
2,আল্লাহর কাউয়ারে আর সাংবাদিকরে তুমি ধ্বংস করে দাও,2
3,টাকলু র দায় সব কিছু,1
4,হে আল্লাহ তুমি তাদেরকে হেদায়েত দান করো। অথবা ...,2
...,...,...
895,ইসলামের উপর যারা আঘাত করছে আল্লা সেইসব মুনাফিক...,2
896,শুধু করস ফায়ার দেন তাতেই আশা করি সব কিছু ঠিক হ...,2
897,ধর্ম যার উৎসবও তার। তবে নাগরিক হিসেবে উৎসব পাল...,0
898,দালাল মিডিয়া তোর গালে কুত্তা দিয়ে মুতামু।৭১টিব...,1


In [23]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate
!pip install -U sentence-transformers

In [24]:
train[train["label"]==0]["text"]

5      বুজলাম না, যারা পবিত্র কোরআনকে অপমান করছে তাদে...
14     হে আমার রব এই জালিমের জুলুম থেকে আমার মুসলমান ...
16     ভাই আপনার ধর্ম নিয়ে রাজনীতি করবেন না আপনারা যে...
18     আমি ভাবতেছি কাল থেকে আমি আন্ডারওয়ার পড়ে স্কু...
20                         হামলাকারীদের গ্রেফতার করা হোক
                             ...                        
873                             মিডিয়া মিথ্যা প্রচার করে
881                     থেকে কোন বিচার এর আসা করা যায় না
891    জনগণ দ্রব্য মূল্যের দাম বৃদ্ধি সমর্থন করে বলেই...
892                         সুন্দর সিদ্ধান্ত নেওয়া হয়েছে
897    ধর্ম যার উৎসবও তার। তবে নাগরিক হিসেবে উৎসব পাল...
Name: text, Length: 300, dtype: object

In [25]:
from sentence_transformers import SentenceTransformer,util
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences_0 = list(train[train["label"]==0]["text"])
sentences_1 = list(train[train["label"]==1]["text"])
sentences_2 = list(train[train["label"]==2]["text"])

# sentences_0 = sentences_0[:300]
# sentences_1 = sentences_1[:300]
# sentences_2 = sentences_2[:300]

#Sentences are encoded by calling model.encode()
embeddings_0 = model.encode(sentences_0)
embeddings_1 = model.encode(sentences_1)
embeddings_2 = model.encode(sentences_2)



In [26]:
embeddings_0.shape

(300, 384)

In [27]:
# test = pd.read_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/data/dev.csv")
test = pd.read_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/data/test_final_phase.csv")
test

,text
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ..."
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা
...,...
2011,নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...
2012,হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...
2013,ভারত ধ্বংস হবে অতি দ্রুত
2014,ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...


In [28]:
np.argmax([1,4,11.12,11.31,3])

3

In [29]:
predictions = []
for index,row in test.iterrows():
  test_embedding = model.encode(row['text'])
  predictions.append(
      np.argmax([
        torch.max(util.cos_sim(test_embedding,embeddings_0)),
        torch.max(util.cos_sim(test_embedding,embeddings_1)),
        torch.max(util.cos_sim(test_embedding,embeddings_2))
      ]

    )
  )


In [30]:
import pandas as pd
d = {
    "text":list(test['text']),
    "label":predictions
}

In [31]:
submission = pd.DataFrame(d)
submission


,text,label
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,2
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,1
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",1
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,2
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,0
...,...,...
2011,নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...,2
2012,হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...,1
2013,ভারত ধ্বংস হবে অতি দ্রুত,2
2014,ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...,2


In [32]:
submission.to_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/submissions/final_phase/submission-7-zero-shot-sbert-sent-sim.csv",index=False)

## Prep Data

https://www.kaggle.com/code/sauravjoshi23/text-classification-using-siamesenet-glove/notebook

In [33]:
# Creating pairs of data for siamese training => label 1 if pairs from same class otherwise 0
df2 = pd.DataFrame(columns=['text1', 'text2', 'label'])

In [34]:
for data in sentences_0:
  data1 = data
  data2 = random.choice(sentences_0)
  data3 = random.choice(sentences_1)
  data4 = random.choice(sentences_2)

  df2.loc[len(df2)] = [data1, data2, 0.9]
  df2.loc[len(df2)] = [data1, data3, 0.1]
  df2.loc[len(df2)] = [data1, data4, 0.1]

In [35]:
for data in sentences_1:
  data1 = data
  data2 = random.choice(sentences_1)
  data3 = random.choice(sentences_0)
  data4 = random.choice(sentences_2)

  df2.loc[len(df2)] = [data1, data2, 0.9]
  df2.loc[len(df2)] = [data1, data3, 0.1]
  df2.loc[len(df2)] = [data1, data4, 0.1]

In [36]:
for data in sentences_2:
  data1 = data
  data2 = random.choice(sentences_2)
  data3 = random.choice(sentences_0)
  data4 = random.choice(sentences_1)

  df2.loc[len(df2)] = [data1, data2, 0.9]
  df2.loc[len(df2)] = [data1, data3, 0.1]
  df2.loc[len(df2)] = [data1, data4, 0.1]

In [37]:
df2

,text1,text2,label
0,"বুজলাম না, যারা পবিত্র কোরআনকে অপমান করছে তাদে...",ছাত্র রাজনীতি দেশ ও শিক্ষা ধংশের মূল,0.9
1,"বুজলাম না, যারা পবিত্র কোরআনকে অপমান করছে তাদে...",৭জন লোক মারা গেচে সেই কথা বললি না কেন ঐই বেইশা...,0.1
2,"বুজলাম না, যারা পবিত্র কোরআনকে অপমান করছে তাদে...","পিডা সালাগো, সালারা আন্দোলনের নামে গাড়ি ভাংগে ...",0.1
3,হে আমার রব এই জালিমের জুলুম থেকে আমার মুসলমান ...,তোমরা কি বোকা ! গেরুয়া পরে আসলে তাতে তোমাদের ক...,0.9
4,হে আমার রব এই জালিমের জুলুম থেকে আমার মুসলমান ...,হেফাজত ও বিএনপি সহ এই চাগুর দল দেশের ৫০তম স্বা...,0.1
...,...,...,...
2695,শুধু করস ফায়ার দেন তাতেই আশা করি সব কিছু ঠিক হ...,আবরার হত্যার বিচার হবে কবে??,0.1
2696,শুধু করস ফায়ার দেন তাতেই আশা করি সব কিছু ঠিক হ...,আপু হচ্ছে বাঘের বাচ্চা। সাবাস।শত সহস্র স্যালুট...,0.1
2697,সেকুলার কান্ট্রি বলে কথা।শয়তান শয়তানের কাজ করত...,নাটক বন্ধ কর। বোন? একে শাদী করতে ও বাধা থাকে ন...,0.9
2698,সেকুলার কান্ট্রি বলে কথা।শয়তান শয়তানের কাজ করত...,শিক্ষা কাকে বলে। শিক্ষার উদ্দেশ্য কি ।,0.1


In [38]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [39]:
train_examples = []
for index,row in df2.iterrows():
  train_examples.append(
      InputExample(
          texts = [
              row["text1"],
              row["text2"]
          ],
          label = row["label"]
      )
  )

len(train_examples)

2700

In [40]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [41]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

In [42]:
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=50, warmup_steps=100,show_progress_bar=True)

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

Iteration:   0%|          | 0/169 [00:00<?, ?it/s]

In [43]:
predictions = []
for index,row in test.iterrows():
  test_embedding = model.encode(row['text'])
  predictions.append(
      np.argmax([
        torch.max(util.cos_sim(test_embedding,embeddings_0)),
        torch.max(util.cos_sim(test_embedding,embeddings_1)),
        torch.max(util.cos_sim(test_embedding,embeddings_2))
      ]

    )
  )


In [44]:
import pandas as pd
d = {
    "text":list(test['text']),
    "label":predictions
}

In [45]:
submission = pd.DataFrame(d)
submission

,text,label
0,বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...,0
1,মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...,0
2,"দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...",1
3,বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...,0
4,বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা,1
...,...,...
2011,নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...,0
2012,হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...,2
2013,ভারত ধ্বংস হবে অতি দ্রুত,2
2014,ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...,0


In [46]:
submission.to_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/submissions/final_phase/submission-8-few-shot-sbert-sent-sim-100-samples.csv",index=False)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
f1_score(y_true=y_test, y_pred=predictions, average='macro')

0.3821598360484922

In [ ]:
model.encode()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
import os

base_directory = "/content/vitd"
subdirectories = ["train", "0", "1", "2"]

# Create the base directory
if not os.path.exists(base_directory):
    os.mkdir(base_directory)

# Create subdirectories
for subdir in subdirectories[1:]:
    subdir_path = os.path.join(base_directory, "train", subdir)
    if not os.path.exists(subdir_path):
        os.makedirs(subdir_path)

print("Directory structure created successfully.")



Directory structure created successfully.


In [ ]:
output_directory = "/content/vitd/train"
for index, row in train.iterrows():
    text = row["text"]
    label = row["label"]

    # Create a filename based on index and label
    filename = f"file_{index}_label_{label}.txt"

    # Create and write to the text file
    file_path = os.path.join(os.path.join(output_directory,str(label)), filename)
    with open(file_path, "w") as f:
        f.write(text)

In [ ]:
import os

directory_path = "/content/vitd/train/0"
directory_path_1 = "/content/vitd/train/1"
directory_path_2 = "/content/vitd/train/2"

# Get a list of all items in the directory
items = os.listdir(directory_path)



# Filter out directories and count the files
file_count = sum(1 for item in items if os.path.isfile(os.path.join(directory_path, item)))

print(f"0 label files count: {file_count}")

items = os.listdir(directory_path_1)
file_count_1 = sum(1 for item in items if os.path.isfile(os.path.join(directory_path_1, item)))

print(f"1 label files count: {file_count_1}")

items = os.listdir(directory_path_2)
file_count_2 = sum(1 for item in items if os.path.isfile(os.path.join(directory_path_2, item)))

print(f"2 label files count: {file_count_2}")

print(f"Total : {file_count + file_count_1 + file_count_2}")

0 label files count: 1389
1 label files count: 922
2 label files count: 389
Total : 2700


## Model

In [ ]:
# !rm -r "/content/vitd/train/.ipynb_checkpoints"


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32 #8 #16
seed = 31
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
 '/content/vitd/train/',
 batch_size=batch_size,
 validation_split=0.2,
 subset='training',
 class_names = ['0','1','2'],
 seed=seed)

Found 2700 files belonging to 3 classes.
Using 2160 files for training.


In [ ]:
class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def build_classifier_model():
 text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
 preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
 encoder_inputs = preprocessing_layer(text_input)
 encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='LaBSE_encoder')
 outputs = encoder(encoder_inputs)
 net = outputs['pooled_output']
 net = tf.keras.layers.Dropout(0.1)(net)
 net = tf.keras.layers.Dense(1, name='classifier')(net)
 return tf.keras.Model(text_input, net)

In [ ]:
tfhub_handle_preprocess="https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
tfhub_handle_encoder="https://tfhub.dev/google/LaBSE/2"

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.1 MB/s eta 0:00:00


In [ ]:
from tensorflow_addons.optimizers import AdamW
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
 [1000], [5e-5,1e-5])
lr = 1 * schedule(step)
wd = lambda: 1e-6 * schedule(step)
optimizer=AdamW(learning_rate=lr,weight_decay=wd)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
classifier_model = build_classifier_model()

In [ ]:
classifier_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
 optimizer=optimizer,
 metrics=tf.keras.metrics.BinaryAccuracy(threshold=0.0))
history = classifier_model.fit(train_ds,
                               epochs=10, batch_size=32)

Epoch 1/10
68/68 [==============================] - 43s 368ms/step - loss: -0.2362 - binary_accuracy: 0.5690
Epoch 2/10
66/68 [============================>.] - ETA: 0s - loss: -2.4170 - binary_accuracy: 0.6283

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/data/dev.csv")
test

,text,label
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,0
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,0
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,0
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,0
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,1
...,...,...
1325,নাটক টা সুন্দর ভাবে সাজিয়েছে আরো কত কিছু দেখত...,1
1326,নোংরা দেশ আর নোংরা জাতি হচ্ছে ভারত এঁরা কি বুঝ...,1
1327,জে ছেলে মারা গেছে ওর কি হবে,0
1328,এরাই নৈরাজ্য সৃষ্টি করছে,1


In [ ]:
X_test = list(test["text"])
y_test = list(test["label"])
encodings = tokenizer(X_test, truncation=True, padding=True) #, max_length=128)

test_dataset = VITD_Dataset(X_test, y_test)
test_dataset

In [ ]:
trainer.save_model("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/models/banglabert_30_epochs_lr_set_V100.pt")

In [ ]:
model_trained = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/models/banglabert_30_epochs_lr_set_V100.pt", config=config)

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification",tokenizer = tokenizer, model=model_trained)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# test.shape[0]
from tqdm import tqdm
predictions = []
for i in tqdm(range(test.shape[0])):
  prediction = classifier(test["text"].iloc[i])
  if prediction[0]['label'] == 'LABEL_0':
    predictions.append(0)
  elif prediction[0]['label'] == 'LABEL_1':
    predictions.append(1)
  if prediction[0]['label'] == 'LABEL_2':
    predictions.append(2)



100%|██████████| 1330/1330 [01:38<00:00, 13.49it/s]


In [ ]:
len(predictions)

1330

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
f1_score(y_true=y_test, y_pred=predictions, average='macro')

0.7602236285532379

In [ ]:
import pandas as pd
d = {
    "text":X_test,
    "label":predictions
}

In [ ]:
submission = pd.DataFrame(d)
submission


,text,label
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,1
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,0
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,0
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,0
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,2
...,...,...
1325,নাটক টা সুন্দর ভাবে সাজিয়েছে আরো কত কিছু দেখত...,1
1326,নোংরা দেশ আর নোংরা জাতি হচ্ছে ভারত এঁরা কি বুঝ...,1
1327,জে ছেলে মারা গেছে ওর কি হবে,0
1328,এরাই নৈরাজ্য সৃষ্টি করছে,1


In [ ]:
submission.to_csv("/content/drive/MyDrive/research/emnlp_blp_2023/task_1/submissions/banglabert_30_epochs_lr_set_V100.csv", index=False)